In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyMuPDFLoader, CSVLoader

from py2neo import Graph, Node, Relationship
from py2neo import NodeMatcher, RelationshipMatcher
import spacy
from tqdm import tqdm
import re

from sentence_transformers import SentenceTransformer

model_path="/home/yuxin.yang/model/moka-ai/m3e-base"
model = SentenceTransformer(model_path)


graph=Graph('http://localhost:7474')
node_matcher = NodeMatcher(graph)

nlp = spacy.load("zh_core_web_trf")

user_query=""

doc=nlp(user_query)
entity_list=[]
for ent in doc.ents:
    if ent not in entity_list:
        #print(ent)
        entity_list.append(ent.text)

print(entity_list)

for ent in entity_list:
    ent_vector = model.encode(ent, normalize_embeddings=False).tolist()
    # query ="""
    # MATCH (p)
    # WHERE p.embedding IS NOT NULL AND gds.similarity.cosine(p.embedding, $query_vector) > 0.8
    # WITH p, gds.similarity.cosine(p.embedding, $query_vector) AS sim
    # ORDER BY sim DESC
    # RETURN p.text AS text, gds.similarity.cosine(p.embedding, $query_vector) AS similarity
    # LIMIT 1
    # """
    # results = graph.run(query, query_vector=ent_vector).data()
    # ent_text=[]
    # for record in results:
    #     #print(record)
    #     print(record['text'], record['similarity'])
    #     ent_text.append(record['text'])
    #     query ="""
    #     MATCH (p)-[r]-(connected_nodes:CONTENT)
    #     WHERE p.text = $text
    #     RETURN connected_nodes.text AS text
    #     LIMIT 300
    #     """
    #     results = graph.run(query, text=record['text']).data()
    #     for record in results:
    #         #print(record)
    #         print(record['text'])


    query ="""
    MATCH (p)-[r]-(connected_node:CONTENT)
    WHERE p.embedding IS NOT NULL AND gds.similarity.cosine(p.embedding, $query_vector) > 0.95
    WITH p, connected_node, gds.similarity.cosine(p.embedding, $query_vector) AS sim
    ORDER BY sim DESC
    RETURN connected_node.text AS text
    LIMIT 300
    """
    results = graph.run(query, query_vector=ent_vector).data()
    all_text=''
    for record in results:
        #print(record)
        print(record['text'])
        all_text+='。'+record['text']




In [ ]:
#利用大语言模型回答
import openai

openai.api_base = ""
openai.api_key = ""


augment_query=""
augment_query+="问题："+ user_query + "\n"
augment_query+="相关资料："+all_text
history=[]
history.append({"role": "user", "content": augment_query})


response = openai.ChatCompletion.create(
            model="",
            messages=history,
            # stream=True,
            temperature=0,
            # stop=["<|im_start|>", "<|im_end|>"],
        )


answer = response.choices[0].message.content

print(answer)

In [ ]:
#利用大语言模型回答
import openai

openai.api_base = ""
openai.api_key = ""


augment_query="\n"
augment_query+="问题："+ user_query + "\n"
augment_query+="相关资料："+all_text
history=[]
history.append({"role": "user", "content": augment_query})


response = openai.ChatCompletion.create(
            model="",
            messages=history,
            # stream=True,
            temperature=0,
            # stop=["<|im_start|>", "<|im_end|>"],
        )


answer = response.choices[0].message.content

print(answer)